In [178]:
import numpy as np 
import pandas as pd
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [179]:
articles = pd.read_csv('C:/Users/SepidehHabibiabad/OneDrive - C-Clear Partners BVBA/Desktop/AI project\AI_project_data/articles.csv')
customers = pd.read_csv('C:/Users/SepidehHabibiabad/OneDrive - C-Clear Partners BVBA/Desktop/AI project\AI_project_data/customers.csv')
sample_submisison = pd.read_csv('C:/Users/SepidehHabibiabad/OneDrive - C-Clear Partners BVBA/Desktop/AI project\AI_project_data/sample_submission.csv')
transactions = pd.read_csv('C:/Users/SepidehHabibiabad/OneDrive - C-Clear Partners BVBA/Desktop/AI project\AI_project_data/transactions_train.csv')

In [180]:
# Adapted from: https://www.kaggle.com/code/paweljankiewicz/hm-create-dataset-samples
# This extracts three sampled datasets, containing 0.1%, 1% and 5% of all users and their transactions, and the associated articles.
for sample_repr, sample in [("01", 0.001), ("1", 0.01), ("5", 0.05)]:
    print(sample)
    customers_sample = customers.sample(int(customers.shape[0]*sample), replace=False)
    customers_sample_ids = set(customers_sample["customer_id"])
    transactions_sample = transactions[transactions["customer_id"].isin(customers_sample_ids)]
    articles_sample_ids = set(transactions_sample["article_id"])
    articles_sample = articles[articles["article_id"].isin(articles_sample_ids)]
    customers_sample.to_csv(f"C:/Users/SepidehHabibiabad/OneDrive - C-Clear Partners BVBA/Desktop/AI project\AI_project_data/customers_sample{sample_repr}.csv.gz", index=False)
    transactions_sample.to_csv(f"C:/Users/SepidehHabibiabad/OneDrive - C-Clear Partners BVBA/Desktop/AI project\AI_project_data/transactions_sample{sample_repr}.csv.gz", index=False)
    articles_sample.to_csv(f"C:/Users/SepidehHabibiabad/OneDrive - C-Clear Partners BVBA/Desktop/AI project\AI_project_data/articles_sample{sample_repr}.csv.gz", index=False)

0.001
0.01
0.05


In [181]:
articles_sample = pd.read_csv('C:/Users/SepidehHabibiabad/OneDrive - C-Clear Partners BVBA/Desktop/AI project\AI_project_data/articles_sample01.csv.gz')
customers_sample = pd.read_csv('C:/Users/SepidehHabibiabad/OneDrive - C-Clear Partners BVBA/Desktop/AI project\AI_project_data/customers_sample01.csv.gz')
transactions_sample = pd.read_csv('C:/Users/SepidehHabibiabad/OneDrive - C-Clear Partners BVBA/Desktop/AI project\AI_project_data/transactions_sample01.csv.gz')

In [182]:
transactions = transactions_sample
customers = customers_sample
articles = articles_sample

In [183]:
transactions['ordered'] = 1

In [184]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2018-09-20,1c1a0113e8b1fa3196aa815a75ee6fa190b523d614f36d...,501820001,0.016932,2,1
1,2018-09-20,1c1a0113e8b1fa3196aa815a75ee6fa190b523d614f36d...,635957001,0.042356,2,1
2,2018-09-20,1c1a0113e8b1fa3196aa815a75ee6fa190b523d614f36d...,585754001,0.025407,2,1
3,2018-09-20,1c1a0113e8b1fa3196aa815a75ee6fa190b523d614f36d...,501820008,0.016932,2,1
4,2018-09-20,2eff272a77e099bec5f1d0b75ed8043767c525bc093db4...,562252010,0.025407,1,1


In [185]:
positive_pairs = list(map(tuple, transactions[['customer_id', 'article_id']].drop_duplicates().values))

In [186]:
real_dates = transactions["t_dat"].unique()
real_customers = transactions["customer_id"].unique()
real_articles = transactions["article_id"].unique()
real_channels = transactions["sales_channel_id"].unique()
article_and_price = transactions[["article_id","price"]].drop_duplicates("article_id").set_index("article_id").squeeze()

In [187]:
num_neg_pos = transactions.shape[0]
print(num_neg_pos)

32336


In [188]:
# Sampling negatives by selecting random users, articles, dates and sales channel:
# Note: This is quite naive. Some articles may not even have been available at the date we are sampling.
random.seed(42)

# Afterwards, we need to remove potential duplicates, so we'll sample too many.
num_neg_samples = int(num_neg_pos * 1.1)

# Sample each of the independent attributes.
neg_dates = np.random.choice(real_dates, size=num_neg_samples)
neg_articles = np.random.choice(real_articles, size=num_neg_samples)
neg_customers = np.random.choice(real_customers, size=num_neg_samples)
neg_channels = np.random.choice(real_channels, size=num_neg_samples)
ordered = np.array([0] * num_neg_samples)
# Assign to every article a real price.
neg_prices = article_and_price[neg_articles].values

In [189]:
neg_transactions = pd.DataFrame([neg_dates, neg_customers, neg_articles, neg_prices, neg_channels, ordered], index=transactions.columns).T

In [190]:
neg_transactions.shape

(35569, 6)

In [191]:
# Remove random negative samples that actually coincide with positives
df = neg_transactions[
    ~neg_transactions.set_index(["customer_id", "article_id"]).index.isin(positive_pairs)
]

# Remove any excess
chosen_neg_transactions = df.sample(num_neg_pos)

In [192]:
# Concat the negative samples to the positive samples:
transactions = pd.concat([transactions, chosen_neg_transactions])
transactions = transactions.merge(customers, how="inner", on='customer_id')
transactions = transactions.merge(articles, how="inner", on='article_id')

In [193]:
######################################################

In [207]:
# transactions_processed = transactions[['customer_id', 'age', 'article_id', 'sales_channel_id', 'price', 'ordered']].copy()
# transactions_processed.head()
transactions_processed = transactions[['customer_id','age','article_id', 'price','t_dat','product_type_no','colour_group_code', 'ordered']].copy()
transactions_processed.head()

,customer_id,age,article_id,price,t_dat,product_type_no,colour_group_code,ordered
0,1c1a0113e8b1fa3196aa815a75ee6fa190b523d614f36d...,21.0,501820001,0.016932,2018-09-20,252,9,1
1,7eb7b96c71ef481db8d921aa979316b46d4b6abc7d02e9...,27.0,501820001,0.016932,2018-12-03,252,9,1
2,d8539d6b469f6d066bb042ce35e5a03a98c5435e6c2347...,51.0,501820001,0.014424,2018-10-24,252,9,1
3,83f8114468be6f2da069d5332c6965c650e3898712853f...,49.0,501820001,0.016932,2019-06-06,252,9,0
4,6c03c3a6182af63b34b7c3be86608f292f46f1d85fd05c...,52.0,501820001,0.016932,2020-04-04,252,9,0


In [160]:
# Does it make sense to label encode?
# Label encoding the customer and article IDs:
# customer_encoder = preprocessing.LabelEncoder()
# article_encoder = preprocessing.LabelEncoder()

In [161]:
# transactions_processed['customer_id'] = customer_encoder.fit_transform(transactions_processed['customer_id'])
# transactions_processed['article_id'] = article_encoder.fit_transform(transactions_processed['article_id'])

In [162]:
transactions_processed.head()

,customer_id,age,article_id,price,t_dat,product_type_no,colour_group_code,ordered
0,03126e45c08e82cd192b32cbddbeba63387e11e5531059...,60.0,378124009,0.06778,2018-09-20,257,73,1
1,03126e45c08e82cd192b32cbddbeba63387e11e5531059...,60.0,378124009,0.06778,2018-09-20,257,73,1
2,85ba782ad67573699594bc726588c3606b2471ab01cbaf...,55.0,378124009,0.06778,2020-08-14,257,73,0
3,1357afdba0b099776afa5663d90b6692b2e44d9973aba0...,34.0,378124009,0.06778,2020-07-19,257,73,0
4,c6427972db94344f8c2f3f6b4a2b8f49429171341b737c...,20.0,378124009,0.06778,2019-11-04,257,73,0


In [208]:
transactions_processed['age'].fillna(int((transactions_processed['age'].mean())), inplace=True)
transactions_processed = transactions_processed.fillna(0)
transactions_processed.isnull().values.any()

False

In [196]:
# most product type that has bought by each user
mode= transactions_processed.groupby(['customer_id'])['product_type_no'].agg(lambda x: pd.Series.mode(x)[0]) 
transactions_processed = transactions_processed.merge(mode, how="inner", on='customer_id')

In [197]:
average = transactions_processed.groupby(['customer_id'])['price'].mean()
transactions_processed = transactions_processed.merge(average, how="inner", on='customer_id')

In [198]:
# favorite color of the costumer
favorite_color = transactions_processed.groupby(['customer_id'])['colour_group_code'].agg(lambda x: pd.Series.mode(x)[0]) 
transactions_processed = transactions_processed.merge(favorite_color, how="inner", on='customer_id')

In [210]:
item_date = transactions.loc[transactions["ordered"] == 1][["customer_id", "article_id", "t_dat"]]
item_date['t_dat'] = pd.to_datetime(item_date["t_dat"])
item_date['recently_bought'] = item_date['t_dat'].max()
item_date['days_difference'] = (item_date['recently_bought'] - item_date['t_dat']).dt.days

In [212]:
popular_month = item_date.loc[item_date["days_difference"] <= 31]
popular_month = last_week_month["article_id"].value_counts().rename_axis("article_id").reset_index(name="popular")
transactions_processed = transactions_processed.merge(popular_month, how="inner", on='article_id')

In [213]:
customer_encoder = preprocessing.LabelEncoder()
article_encoder = preprocessing.LabelEncoder()
transactions_processed['customer_id'] = customer_encoder.fit_transform(transactions_processed['customer_id'])
transactions_processed['article_id'] = article_encoder.fit_transform(transactions_processed['article_id'])

In [214]:
transactions_processed = transactions_processed.drop(['t_dat'], axis=1)

In [215]:
transactions_processed.head()

,customer_id,age,article_id,price,product_type_no,colour_group_code,ordered,popular
0,78,21.0,223,0.008458,253,9,1,1
1,378,52.0,223,0.008458,253,9,1,1
2,420,24.0,223,0.006915,253,9,1,1
3,552,51.0,223,0.008458,253,9,1,1
4,664,38.0,223,0.008458,253,9,1,1


In [216]:
X_train, X_test, y_train, y_test = train_test_split(transactions_processed.drop('ordered', axis=1), transactions_processed['ordered'], test_size=0.10, random_state=42)

In [217]:
X_train.head()

,customer_id,age,article_id,price,product_type_no,colour_group_code,popular
637,402,26.0,44,0.050831,272,9,1
1151,0,31.0,21,0.025407,259,10,1
619,653,28.0,265,0.022864,298,9,1
1319,283,40.0,20,0.030492,306,10,1
602,110,33.0,0,0.011288,273,9,1


In [218]:
baseline = LogisticRegression(random_state=42)
baseline = baseline.fit(X_train, y_train)

In [219]:
baseline.score(X_test, y_test)

0.7307692307692307

In [206]:
predictions = baseline.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.53      0.45      0.49      3284
           1       0.51      0.59      0.55      3184

    accuracy                           0.52      6468
   macro avg       0.52      0.52      0.52      6468
weighted avg       0.52      0.52      0.52      6468

